# Second Quantized Hamiltonian

In [1]:
import numpy as np

from classiq import Model, construct_chemistry_model, execute, show, synthesize
from classiq.applications.chemistry import (
    ChemistryExecutionParameters,
    FermionicOperator,
    HamiltonianProblem,
    HEAParameters,
    SummedFermionicOperator,
)
from classiq.builtin_functions import HardwareEfficientAnsatz
from classiq.execution import OptimizerType

In [2]:
hamiltonian = SummedFermionicOperator(
    op_list=[
        (FermionicOperator(op_list=[("+", 0), ("-", 0)]), 0.2),
        (FermionicOperator(op_list=[("-", 1), ("-", 1)]), 0.3),
        (FermionicOperator(op_list=[("-", 2), ("-", 2)]), 0.4),
        (FermionicOperator(op_list=[("-", 3), ("-", 3)]), 0.5),
        (FermionicOperator(op_list=[("+", 0), ("+", 1), ("-", 1), ("-", 0)]), -0.1),
        (FermionicOperator(op_list=[("+", 2), ("+", 3), ("-", 2), ("-", 3)]), -0.3),
    ]
)
ham_problem = HamiltonianProblem(hamiltonian=hamiltonian, num_particles=[1, 1])

print(ham_problem.hamiltonian)

0.2 * a⁺₀a₀ + 
0.3 * a₁a₁ + 
0.4 * a₂a₂ + 
0.5 * a₃a₃ + 
-0.1 * a⁺₀a⁺₁a₁a₀ + 
-0.3 * a⁺₂a⁺₃a₂a₃


In [3]:
hwea_params = HEAParameters(
    num_qubits=4,
    connectivity_map=[(0, 1), (1, 2), (2, 3)],
    reps=3,
    one_qubit_gates=["x", "ry"],
    two_qubit_gates=["cx"],
)

qmod = construct_chemistry_model(
    chemistry_problem=ham_problem,
    use_hartree_fock=False,
    ansatz_parameters=hwea_params,
    execution_parameters=ChemistryExecutionParameters(
        optimizer=OptimizerType.COBYLA,
        max_iteration=100,
        initial_point=None,
    ),
)

In [4]:
with open("second_quantized_hamiltonian.qmod", "w") as f:
    f.write(qmod)

In [5]:
qprog = synthesize(qmod)
show(qprog)

Opening: https://platform.classiq.io/circuit/476a5337-6155-4261-8cc7-e47ee5c7516c?version=0.34.0.dev0%2B124706fb0e


In [6]:
results = execute(qprog).result()
chemistry_result = results[0].value
chemistry_result.energy

0.0008999999999999625